In [111]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from fuzzywuzzy import fuzz

# Inisialisasi modul NLTK
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dennisherdi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dennisherdi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [112]:
df = pd.read_csv('cb_orgs_202302220956.csv')

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17811 entries, 0 to 17810
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    17811 non-null  object
dtypes: object(1)
memory usage: 139.3+ KB


In [114]:
df.head()

,name
0,Meridian Entertainment Group
1,Bishop Canevin High School
2,Everside Health
3,Spire Group
4,Applied BioCode


In [115]:
print('Number of missing values per feature')
df.isna().sum()

Number of missing values per feature


name    0
dtype: int64

In [116]:
print(f"Is there any duplicated rows? \n{df.duplicated().any()}") #melihat duplikat dari data
print(f"\nHow many? \n{df.duplicated().sum()} rows")

Is there any duplicated rows? 
True

How many? 
246 rows


In [117]:
df.drop_duplicates(subset=None, keep='first', inplace = True, ignore_index=False) #menghaopus data duplikat

In [118]:
import re 
import string

In [119]:
df1 = df

In [120]:
def remove_punct(text): #menghapus tanda baca
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

df1['name'] = df['name'].apply(lambda x: remove_punct(x))

In [121]:
# # fungsi clean digunakan membersihkan simbol atau frasa yang tidak diperlukan
# def cleanOrg(org):
#     org = org.lower()
#     org = re.sub(r'\d+', '', org) # remove digits
#     org = re.sub(r'”|–|“|’', '', org) # remove digits
#     org = re.sub(r'\n', '', org) # remove enter to 1 space
#     org = re.sub(r'tbk', ' ', org)
#     org = re.sub(r'pt', ' ', org)
#     org = re.sub(r'\s+', ' ', org)# remove extra spaces
    
    
#     return org

# # mengaplikasikan proses cleaning
# df1['name'] = df1.name.astype(str).apply(cleanOrg)

In [122]:
df1['name']

0        Meridian Entertainment Group
1          Bishop Canevin High School
2                     Everside Health
3                         Spire Group
4                     Applied BioCode
                     ...             
17805                         Rollins
17806              SBA Communications
17807                          Deluxe
17808                   Herc Holdings
17809                        SPX FLOW
Name: name, Length: 17565, dtype: object

In [123]:
# Menyiapkan data
documents = df1['name']

# Proses tokenisasi dan lemmatisasi
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]
    return tokens

processed_documents = [preprocess_text(document) for document in documents]


In [124]:
# Fuzzy search
query = 'ojk'
results = []
for i, document in enumerate(processed_documents):
    if all(isinstance(token, str) for token in document) and len(document) > 0:
        match_score = max([fuzz.ratio(query, token) for token in document])
        if match_score > 70:
            # Store the match score and document in a tuple
            result = (match_score, document)
            results.append(result)

# Sort the results by match score in descending order
sorted_results = sorted(results, key=lambda x: x[0], reverse=True)[:5]
# Print the sorted results
for result in sorted_results:
    match_score = result[0]
    document = result[1]
    print(f"Query '{query}' found in document {document} with match score {match_score}")


Query 'ojk' found in document ['jk', 'medicine'] with match score 80
Query 'ojk' found in document ['jk', 'gadzama', 'llp'] with match score 80
Query 'ojk' found in document ['jk', 'recycling'] with match score 80
Query 'ojk' found in document ['ok', 'computadores'] with match score 80
Query 'ojk' found in document ['gojek'] with match score 75


In [125]:
sorted_results

[(80, ['jk', 'medicine']),
 (80, ['jk', 'gadzama', 'llp']),
 (80, ['jk', 'recycling']),
 (80, ['ok', 'computadores']),
 (75, ['gojek'])]